## 3 MA. MNL. TP 1. Factorisation de matrices

# 1. Décompositions LU et de Cholesky d'une matrice


**(a)** Implémenter une fonction $\texttt{decompoLU(A)}$ qui prend en argument une matrice $A$ et qui retourne les matrices L et U de la décomposition sous le format $\texttt{[L,U]}$. On se placera dans le cadre de la CNS d'existence et d'unicité de la décomposition, donnée au théorème 4 du cours (diapositive 66).

**(b)** A l'aide de la fonction $\texttt{decompoLU(A)}$, donner la décomposition LU de la matrice
$$
A=
\begin{pmatrix}
2 & -1 & 0 \\
-1 & 2 & -1 \\
0 & -1 & 3 \\
\end{pmatrix}
$$

**(c)** Utiliser la décomposition pour résoudre le système $Ax=b$ avec $b=(1,2,3)^T$ et vérifier si l'implémentation est correcte en comparant le résultat à celui renvoyé par $\texttt{solve(A,b)}$. Il faudra programmer deux fonctions pour les phases de descente et de remontée des systèmes triangulaires associés.

**(d)** Trouver la décomposition de Cholesky de $A$ à partir de la décomposition LU. Là aussi, on programmera une fonction $\texttt{decompoCHOLESKY}$ qui calcule la décomposition de Cholesky d'une matrice vérifiant les hypothèses précisées dans le cours.

**(e)** Vérifier numériquement la résolution directe des systèmes linéaires de l'exercice 7 du TD, Liste 2.







In [4]:
import numpy as np

In [54]:
def eliminationGauss(A,k):
    """Fonction qui realise une étpae de l'algorithme de Gauss
    
    Prends en parametre une matrice A dont les pivots ne sont jamais nul
    et un entier k qui correspond à laquelle appliquer une etape de l'algo de Gauss
    
    Retourne La nouvelle matrice apres avoir subit une etape de Gauss sur sa ligne k
    et une liste contenant les coefficients appliquer à chaque ligne inferieur ou egale a k"""
    n=A.shape[0]
    coefLik = []
    for i in range(k+1,n):
        coef = A[i,k]/A[k,k]
        coefLik.append(coef)
        for j in range(k,n):
            A[i,j] = A[i,j] - coef*A[k,j]
    
    return A,coefLik

def verifSousMatrice(A):
    """Prends en paramètre une matrice carrée A
    
    Retourne un booleen sur la condition nécessaire et suffisante pour la décomposition LU

    Test si toutes les sous-matrices de A ont un déterminant non-nul"""
    result = True
    for i in range(A.shape[0]):
        if np.linalg.det(A[:i,:i])==0:
            result = False
    return result

def decompLU(A):
    """Realise la décomposition LU d'une matrice
    
    Prends en parametre une matrice carrée A
    
    Rtourne les matrices U et L de la décompoition, si cette dernière est unique"""
    if not verifSousMatrice(A):
        raise Exception("La matrice A ne satisfait pas les conditions d'existence et d'unicité de la décomposition LU")
    n=A.shape[0]
    L=[]
    for k in range(0,n):
        A,coefLik = eliminationGauss(A,k)
        l=[]
        for _ in range(0,k):
            l.append(0)
        l.append(1)
        for c in coefLik:
            l.append(c)
        L.append(l)
    L=np.array(L)
    L=L.T
    return A,L

In [55]:
A=np.array([[3,-1,0],
            [-1,2,-1],
            [0,-1,3]], dtype=float)

U,L=decompLU(A.copy())
print("U=\n",U)
print("L=\n",L)


U=
 [[ 3.         -1.          0.        ]
 [ 0.          1.66666667 -1.        ]
 [ 0.          0.          2.4       ]]
L=
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.         -0.6         1.        ]]


In [56]:

def resoudreTriInf(L,b):
    """Fonction qui resoud un systeme triangulaire inférieur
    
    Elle prend en paramètre une matrice triangulaire inferieur L, et un vecteur colonne b
    
    Elle retourne la solution x de l'équation Lx=b"""
    n=L.shape[0]
    y=[]
    y.append((b[0]/L[0,0]))
    for i in range(1,n):
        s=0
        for j,yi in enumerate(y):
            s+=yi*L[i,j]
        y.append((b[i]-s)/L[i,i])
        
    return np.array(y)

def resoudreTriSup(U,b):
    """Fonction qui resoud un systeme triangulaire supérieur
    
    Elle prend en paramètre une matrice triangulaire supérieur U, et un vecteur colonne b
    
    Elle retourne la solution x de l'équation Ux=b"""
    n=U.shape[0]
    y=np.zeros((1,n))
    y[0,n-1]=b[n-1]/U[n-1,n-1]
    for i in range(1,n):
        s=0
        for j,yi in enumerate(y[0,:]):
            s+=yi*U[n-1-i,j]
        y[0,n-1-i]=(b[n-1-i]-s)/U[n-1-i,n-1-i]

    return np.transpose(y)

def resolutionLU(L,U,b):
    """Fonction qui résoud un système à partir de sa décomposition LU
    
    Elle prend en parametre les matrices L et U et le vecteur colonne b
    
    Elle retourne x solution de l'équation LUx=b"""
    y = resoudreTriInf(L,b)
    print("y=\n",y)
    x = resoudreTriSup(U,y)
    return x

In [57]:
b = np.array([[1],
              [2],
              [3]],dtype=float)
x = resolutionLU(L,U,b)
print("x=\n",x)
print("Ax=\n",A@x)

y=
 [[1.        ]
 [2.33333333]
 [4.4       ]]
x=
 [[1.16666667]
 [2.5       ]
 [1.83333333]]
Ax=
 [[1.]
 [2.]
 [3.]]


C:\Users\Thoma\AppData\Local\Temp\ipykernel_13308\2399190758.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[0,n-1]=b[n-1]/U[n-1,n-1]
C:\Users\Thoma\AppData\Local\Temp\ipykernel_13308\2399190758.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[0,n-1-i]=(b[n-1-i]-s)/U[n-1-i,n-1-i]


In [58]:
def decompCholesky(A):
    if not np.all(A==A.T):
        raise Exception("La matrice n'est pas symétrique")
    if not verifSousMatrice:
        raise Exception("La matrice n'est pas définie positive")
    U,L=decompLU(A)
    print("U:\n",U)
    D=np.diag(U)
    F=np.zeros(U.shape)
    for i in range(0,D.shape[0]):
        F[i,i]=np.sqrt(D[i])
    print("F:\n",F)
    G=L@F
    return G,np.transpose(G)

In [59]:
A=np.array([[4,0,2,0],
           [0,4,0,2],
           [2,0,5,0],
           [0,2,0,5]])

G,Gt=decompCholesky(A)
print("G:\n",G)
print("verif\n",G@Gt)

U:
 [[4 0 2 0]
 [0 4 0 2]
 [0 0 4 0]
 [0 0 0 4]]
F:
 [[2. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]
G:
 [[2. 0. 0. 0.]
 [0. 2. 0. 0.]
 [1. 0. 2. 0.]
 [0. 1. 0. 2.]]
verif
 [[4. 0. 2. 0.]
 [0. 4. 0. 2.]
 [2. 0. 5. 0.]
 [0. 2. 0. 5.]]


# 2. Orthonormalisation de Gram-Schmidt et lien avec la décomposition QR
Soit $\mathcal{B}=\{\vec{x}_j\}_{j=1}^m$ une famille de vecteurs linéairement indépendants dans $\mathbb{R}^n$, avec $1\leq m \leq n$. En partant de $\mathcal{B}$, le procédé d'orthonormalisation de Gram--Schmidt permet de construire une famille $\mathcal{Q}=\{\vec{q}_j\}_{j=1}^m$ de vecteurs orthonormaux définis comme suit. Le premier vecteur est
$$
\vec{q}_1=\frac{\vec{x}_1}{\Vert \vec{x}_1 \Vert}.
$$
Ensuite, pour $1<j\leq m$, on pose
$$
\vec{w}_{j}=\vec{x}_{j}-\sum_{i=1}^{j-1}\left(\vec{x}_{j},\vec{q}_i\right)\vec{q}_i
$$
puis
$$
\vec{q}_{j} = \frac{\vec{w}_{j}}{\Vert \vec{w}_{j}\Vert}.
$$

**(a)** Écrire une fonction $\texttt{gramschmidt(B)}$, prenant comme paramètre d'entrée une matrice $B\in\mathbb{R}^{n\times m}$ ayant pour colonnes les $m$ vecteurs de $\mathcal{B}$ et retournant une matrice $Q$ ayant pour colonnes les $m$ vecteurs de la famille $\mathcal{Q}$. Au début de la fonction, penser à tester 
si la famille $\mathcal{B}$ donnée en entrée est bien libre.

**(b)** On pose $\varepsilon=1$. Orthonormaliser avec la fonction $\texttt{gramschmidt}$ la famille
$$
\mathcal{B}=\left\{\begin{pmatrix}1\\\varepsilon\\0\\0\end{pmatrix},\begin{pmatrix}1\\0\\\varepsilon\\0\end{pmatrix},\begin{pmatrix}1\\0\\0\\\varepsilon\end{pmatrix}\right\},
$$
puis vérifier que les vecteurs obtenus sont bien orthogonaux deux à deux. Que constate-t-on quand $\varepsilon=10^{-8}$? 

**(c)** Mettre en relation la méthode d'orthonormalisation de Gram-Schmidt avec la décomposition QR d'une matrice.

**(d)** Approfondissement : à l'aide du document "modified_Gram-Schmidt.pdf" déposé sur Moddle (source : G. Legendre, *Cours de méthodes numériques*, Université Paris Dauphine, mai 2021) implémenter l'algorithme de factorisation QR d'une matrice inversible par le procédé d'orthonormalisation de Gram-Schmidt (**algorithme 13**), lire la remarque qui suit sur les erreurs d'arrondi, et implémenter la version modifiée (**algorithme 14**). Est-ce que cette version est plus stable que la version classique ?


In [55]:
def gramschmidt(B):
    if not np.linalg.matrix_rank(B)==B.shape[0]:
        raise Exception("La famille B n'est pas libre")
    Q=np.zeros(B.shape)
    Q[0,:]=B[0,:]/np.linalg.norm(B[0,:],ord=2)
    for j in range(1,B.shape[0]):
        w=B[j,:]
        for i in range(0,j):
            w -= np.dot(B[j,:],Q[i,:])*Q[i,:]
        Q[j,:] = w/np.linalg.norm(w,ord=2)
    return Q

In [56]:
def verifOthogonale(Q):
    n=Q.shape[0]
    erreur = np.zeros(n,dtype=float)
    for i in range(0,n-1):
        for j in range(i+1,n):
            erreur[i+j-1]=abs(np.dot(Q[i,:],Q[j,:]))
    print(erreur)
    print("Erreur totale :",np.sum(erreur))

In [57]:
e=10^-8
B=np.array([[1,e,0,0],
           [1,0,e,0],
           [1,0,0,e]],dtype=float)
Q=gramschmidt(B)
print(Q)
verifOthogonale(B)

[[ 0.07124705 -0.9974587   0.          0.        ]
 [ 0.0708863   0.00506331 -0.99747155  0.        ]
 [ 0.07052919  0.0050378   0.0050378  -0.99748427]]
[3.33066907e-16 4.44089210e-16 0.00000000e+00]
Erreur totale : 7.771561172376096e-16
